# 0.0 IMPORTS

In [1]:
import re
import warnings
import sweetviz
import IPython
import pickle

import pandas            as pd
import numpy             as np
import lightgbm          as lgb
import xgboost           as xgb
import seaborn           as sns

from category_encoders       import TargetEncoder
from sklearn.preprocessing   import OneHotEncoder
from sklearn.ensemble        import ExtraTreesClassifier, RandomForestClassifier, StackingClassifier
from scipy                   import stats
from unidecode               import unidecode
from catboost                import CatBoostClassifier
from skopt                   import gp_minimize
from sklearn.model_selection import cross_val_score, KFold

from sklearn                 import model_selection   as ms
from sklearn                 import metrics           as m
from matplotlib              import pyplot            as plt

C:\Users\mathe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 0.1 AUX FUNCTIONS

In [2]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

def ml_error( model_name, ytest, yhat ):
    f1 = m.f1_score( ytest, yhat )

    return pd.DataFrame( {'Model name': model_name,
                          'F1': f1 }, index=[0] )

## 0.2 READ DATASETS

In [4]:
df = pd.read_csv('datasets/train.csv')

# df_dados_estado = pd.read_csv( "datasets publicos/Dados Estado Simplificado.csv" )
# df_dados_municipio = pd.read_excel( "datasets publicos/Dados Município Simplificado.xlsx", sheet_name="Dados" )
# df_pib = pd.read_excel( "datasets publicos/PIB Municípios.xlsx" )
# df_ibge = pd.read_excel( "datasets publicos/IBGE.xlsx", sheet_name="Planilha1" )

### 0.2.2 MERGE PUBLIC DATASETS

In [4]:
# df_dados_estado.rename( columns={ 'População':'populacao_estado','Matrículas':'matriculas_estado',
#                                   'Categoria do Quadrante':'categoria_quadrante_estado','Insumos do Ioeb':'insumos_ioeb_estado',
#                                   'Ioeb':'ioeb_estado','Resultados do Ioeb':'resultados_ioeb_estado'}, inplace=True)
                                  
# df_dados_municipio.rename( columns={ 'População':'populacao_cidade','Total de Matrículas':'matriculas_cidade',
#                                   'Categoria do Quadrante':'categoria_quadrante_cidade','Insumos do Ioeb':'insumos_ioeb_cidade',
#                                   'Ioeb':'ioeb_cidade','Resultados do Ioeb':'resultados_ioeb_cidade','Região':'regiao' }, inplace=True)

# for col in df_dados_estado.columns:
#     if df_dados_estado[f'{col}'].dtypes != "int64" and df_dados_estado[f'{col}'].dtypes != "float64":
#         df_dados_estado[f'{col}'] = df_dados_estado[f'{col}'].apply(lambda x: x.replace( ",", "" ) if type(x) is not float else x )
#         df_dados_estado[f'{col}'] = df_dados_estado[f'{col}'].apply(lambda x: x.replace( ".", "" ) if type(x) is not float else x )

# for col in df_dados_municipio.columns:
#     if df_dados_municipio[f'{col}'].dtypes != "int64" and type(df_dados_municipio[f'{col}']) != "float64":
#         df_dados_municipio[f'{col}'] = df_dados_municipio[f'{col}'].apply(lambda x: x.replace( ",", "" ) if type(x) is not float else x )
#         df_dados_municipio[f'{col}'] = df_dados_municipio[f'{col}'].apply(lambda x: x.replace( ".", "" ) if type(x) is not float else x )

# # merge df users com df dados estado
# df_dados_estado['Estado'] = df_dados_estado['Estado'].apply( lambda x: unidecode(x).upper() )
# df['region'] = df['region'].apply( lambda x: unidecode(x).upper() if type(x) is not float else x )

# df = df.merge( df_dados_estado, how='left', left_on='region', right_on='Estado' )
# df.drop( columns=['Estado','Ano do Ioeb','Código da UF','Unidade da Federação','Região'], inplace=True )

# # merge df users com df dados municipio
# df_dados_municipio['Nome do Município'] = df_dados_municipio['Nome do Município'].apply( lambda x: unidecode(x).upper() )
# df['city'] = df['city'].apply( lambda x: unidecode(x).upper() if type(x) is not float else x )

# df = df.merge( df_dados_municipio, how='left', left_on='city', right_on='Nome do Município' )
# df.drop( columns=['Código do Município','Código da UF','Unidade da Federação','Nome do Município','Ano do Ioeb'], inplace=True )

# features_to_float = ['ioeb_estado', 'insumos_ioeb_estado', 'resultados_ioeb_estado', 'populacao_estado', 
# 'ioeb_cidade', 'insumos_ioeb_cidade', 'resultados_ioeb_cidade', 'matriculas_cidade', 'populacao_cidade']
# df[features_to_float] = df[features_to_float].astype(float)

In [5]:
# df_pib['Município'] = df_pib['Município'].apply( lambda x: x.split('(')[0] )
# df_pib['Município'] = df_pib['Município'].apply( lambda x: unidecode(x).upper().strip() )

# df = df.merge( df_pib, how='left', left_on='city', right_on='Município' )
# df.drop( columns=['Município'], inplace=True )

In [6]:
# df_ibge = df_ibge[['Município','IDHM','IDHM_E','IDHM_L','IDHM_R','I_ESCOLARIDADE','I_FREQ_PROP','TRABPUB','RENOCUP','GINI','T_FLSUPER',
# 'T_ANALF18M','E_ANOSESTUDO','ESPVIDA']]
# df_ibge['Município'] = df_ibge['Município'].apply( lambda x: unidecode(x).upper().strip() )

# df = df.merge( df_ibge, how='left', left_on='city', right_on='Município' )
# df.drop( columns=['Município'], inplace=True )

# 1.0 DATA DESCRIPTION

## 1.1 Shape

In [5]:
print( f"df users rows: {df.shape[0]}" )
print( f"df users columns: {df.shape[1]}" )

df users rows: 9500
df users columns: 17


In [6]:
df.head()

,id_cliente,idade,saldo_atual,divida_atual,renda_anual,valor_em_investimentos,taxa_utilizacao_credito,num_emprestimos,num_contas_bancarias,num_cartoes_credito,dias_atraso_dt_venc,num_pgtos_atrasados,num_consultas_credito,taxa_juros,investe_exterior,pessoa_polit_exp,limite_adicional
0,1767,21,278.172008,2577.05,24196.89636,104.306544,31.038763,6,5,7,21,14,9,15,Não,Não,Negar
1,11920,40,268.874152,2465.39,19227.37796,69.858778,36.917093,5,8,5,40,23,10,18,Não,Não,Negar
2,8910,36,446.643127,1055.29,42822.28223,134.201478,34.561714,0,3,6,26,13,3,15,Sim,Não,Negar
3,4964,58,321.141267,703.05,51786.82600,297.350067,31.493561,0,3,7,12,7,2,1,Sim,Não,Negar
4,10100,35,428.716114,891.29,44626.85346,134.201478,28.028887,2,8,7,24,10,8,20,Sim,Não,Negar


## 1.2 Check NA

In [7]:
df.isna().sum()

id_cliente                 0
idade                      0
saldo_atual                0
divida_atual               0
renda_anual                0
valor_em_investimentos     0
taxa_utilizacao_credito    0
num_emprestimos            0
num_contas_bancarias       0
num_cartoes_credito        0
dias_atraso_dt_venc        0
num_pgtos_atrasados        0
num_consultas_credito      0
taxa_juros                 0
investe_exterior           0
pessoa_polit_exp           0
limite_adicional           0
dtype: int64

### 1.2.1 Fill NA

## 1.3 Data Types

In [9]:
df.dtypes

### 1.3.1 Change Dtypes

In [ ]:
# df['data'] = pd.to_datetime( df['data'] )

# 2.0 FEATURE ENGINEERING

In [8]:
df2 = df.copy()

In [11]:
# # year
# df2['year'] = df2['created_at'].dt.year

# # month
# df2['month'] = df2['created_at'].dt.month

# # day
# df2['day'] = df2['created_at'].dt.day

# # week of year 
# df2['week_of_year'] = df2['created_at'].dt.isocalendar().week

# # week of year 
# df2['day_of_week'] = df2['created_at'].dt.weekday

# # hour
# df2['hour'] = df2['created_at'].dt.hour

# # minute
# df2['minute'] = df2['created_at'].dt.minute

# # second
# df2['second'] = df2['created_at'].dt.second

# # nivel de concentracao
# df2['concentracao'] = df2['row'].apply( lambda x: 'pomodoro1' if x <=10 else 
#                                                               'pomodoro2' if x > 10 and x <= 20 else
#                                                               'pomodoro3' if x > 20 and x <= 30 else
#                                                               'pomodoro4' if x > 30 and x <= 40 else 
#                                                               'pomodoro5' if x > 40 and x <= 50 else 
#                                                               'pomodoro6' if x > 50 and x <= 60 else 
#                                                               'pomodoro7' if x > 60 and x <= 70 else 
#                                                               'pomodoro8' if x > 70 and x <= 80 else 
#                                                               'pomodoro9' if x > 80 and x <= 90 else 
#                                                               'pomodoro10' )

# # letras_recorrentes
# letras_recorrentes = ['A','E']
# df2['letras_recorrentes'] = df2['right_answer'].apply( lambda x: 'recorrente' if x in letras_recorrentes else 'nao recorrente'  )

# 3.0 DATA FILTERING

In [9]:
df3 = df2.copy()

# 4.0 EDA

In [10]:
df4 = df3.copy()

# sweetviz report
# report = sweetviz.analyze( df, "limite_adicional" )
# report.show_html()

# 5.0 DATA PREPARATION

In [11]:
df5 = df3.copy()

In [16]:
# limite adicional
df5['limite_adicional'] = df5['limite_adicional'].apply( lambda x: 0 if x == 'Negar' else 1 )

# target encoder
te = TargetEncoder()

df5['investe_exterior'] = te.fit_transform( df5['investe_exterior'], df5['limite_adicional'] )
df5['pessoa_polit_exp'] = te.fit_transform( df5['pessoa_polit_exp'], df5['limite_adicional'] )

# # natural variable transformation
# # day
# df5['day_sin'] = df5['day'].apply( lambda x: np.sin( x * ( 2 * np.pi/30 ) ) )
# df5['day_cos'] = df5['day'].apply( lambda x: np.cos( x * ( 2 * np.pi/30 ) ) )

# # day of week
# df5['day_of_week_sin'] = df5['day_of_week'].apply( lambda x: np.sin( x * ( 2 * np.pi/7 ) ) )
# df5['day_of_week_cos'] = df5['day_of_week'].apply( lambda x: np.cos( x * ( 2 * np.pi/7 ) ) )

# # month
# df5['month_sin'] = df5['month'].apply( lambda x: np.sin( x * ( 2 * np.pi/12 ) ) )
# df5['month_cos'] = df5['month'].apply( lambda x: np.cos( x * ( 2 * np.pi/12 ) ) )

# # week of year
# df5['week_of_year_sin'] = df5['week_of_year'].apply( lambda x: np.sin( x * ( 2 * np.pi/52 ) ) )
# df5['week_of_year_cos'] = df5['week_of_year'].apply( lambda x: np.cos( x * ( 2 * np.pi/52 ) ) )

# # hour
# df5['hour_sin'] = df5['hour'].apply( lambda x: np.sin( x * ( 2 * np.pi/24 ) ) )
# df5['hour_cos'] = df5['hour'].apply( lambda x: np.cos( x * ( 2 * np.pi/24 ) ) )

# # quarter
# df5['minute_sin'] = df5['minute'].apply( lambda x: np.sin( x * ( 2 * np.pi/60 ) ) )
# df5['minute_cos'] = df5['minute'].apply( lambda x: np.cos( x * ( 2 * np.pi/60 ) ) )

# # quarter
# df5['second_sin'] = df5['second'].apply( lambda x: np.sin( x * ( 2 * np.pi/60 ) ) )
# df5['second_cos'] = df5['second'].apply( lambda x: np.cos( x * ( 2 * np.pi/60 ) ) )

# df5.drop( columns=['day','day_of_week','month','week_of_year','year','hour'], inplace=True )

# 6.0 FEATURE SELECTION

In [15]:
# # model
# forest = ExtraTreesClassifier( n_jobs=-1 )

# # training
# X = df5.copy()
# Y = df5['limite_adicional'].values

# x_train_fselection, x_val, y_train_fselection, y_val = ms.train_test_split( X, Y, test_size=0.5, random_state=42 )

# forest.fit( x_train_fselection, y_train_fselection )

In [16]:
# importances = forest.feature_importances_
# std = np.std( [tree.feature_importances_ for tree in forest.estimators_], axis=0 )
# indices = np.argsort( importances )[::-1]

# # print the feature ranking
# df = pd.DataFrame()

# print( 'Feature Ranking:\n' )
# for i, j in zip( x_train_fselection,forest.feature_importances_ ):
#     aux = pd.DataFrame( {'feature': i, 'importance': j}, index=[0] )
#     df = pd.concat( [df, aux], axis=0 )
    
# print( df.sort_values( 'importance', ascending=False ) ) 

# # plot the impurity-based feature importances of the forest
# plt.figure(figsize=(10,5))
# plt.title( 'Feature importances' )
# plt.bar( range( x_train_fselection.shape[1] ), importances[indices], color='r', yerr=std[indices], align='center' )
# plt.xticks( range(x_train_fselection.shape[1]), indices )
# plt.xlim( [-1, x_train_fselection.shape[1]] )
# plt.show()

# 7.0 MACHINE LEARNING

In [18]:
columns_selected = []

X = df5.drop(columns=['limite_adicional'])
Y = df5['limite_adicional'].copy()

X_train, X_val, y_train, y_val = ms.train_test_split( X, Y, test_size=0.2, random_state=42 )

## 7.1 LightGBM

In [19]:
# model
model_lgb = lgb.LGBMClassifier( n_jobs=-1 ).fit( X_train, y_train )

# prediction
yhat_lgb = model_lgb.predict( X_val )

# performance
model_lgb_results = ml_error( 'LightGBM',  y_val, yhat_lgb )
model_lgb_results

,Model name,F1
0,LightGBM,0.543119


### 7.1.1 Cross Validation

In [3]:
# # cross validation
# kfold = KFold(n_splits=7, random_state=1, shuffle=True)
# cv = cross_val_score(model_lgb, X, Y, scoring='f1', cv=kfold, n_jobs=-1, error_score='raise' )
# print( "{} +/- {}" ).format( np.mean(cv), np.std(cv))

## 7.2 XGBoost

In [ ]:
# # model
# model_xgb = xgb.XGBClassifier( n_jobs=-1 ).fit( X_train, y_train )

# # prediction
# yhat_xgb = model_xgb.predict( X_val )

# # performance
# model_xgb_results = ml_error( 'XGBoost',  y_val, yhat_xgb )
# model_xgb_results

### 7.2.1 Cross Validation

## 7.3 CatBoost

In [ ]:
# # model
# model_cb = CatBoostClassifier().fit( X_train, y_train )

# # prediction
# yhat_cb = model_cb.predict( X_val )

# # performance
# model_cb_results = ml_error( 'CatBoost',  y_val, yhat_cb)
# model_cb_results

### 7.3.1 Cross Validation

## 7.4 Stacking Classifier

In [ ]:
# estimators_list = [ ('lgbm', model_lgb), ('xgboost', model_xgb), ('catboost', model_cb) ]

# # Build stack model
# stack_model = StackingClassifier( estimators = estimators_list, n_jobs=-1, verbose=True ).fit( X_train, y_train )

# # prediction
# yhat_stack = stack_model.predict( X_val )

# # performance
# stack_model_results = ml_error( 'Stacking', y_val, yhat_stack )
# stack_model_results

# 8.0 Bayesian Optimization

## 8.1 LGBM

### 8.1.1 Normal

In [1]:
# search_space = [(100, 1500), #name = 'n_estimators'), 
#                 (1, 20), #name = 'max_depth'), 
#                 (0.001, 0.1, 'log-uniform'),#, name = 'learning_rate'),
#                 (2, 128), #name = 'num_leaves'),
#                 (1, 100), #name = 'min_child_samples'),
#                 (0.05, 1.0), #name = 'subsample'),
#                 (0.15, 1.0) #name = 'colsample_bytree')]
# ]

# def treinar_modelo( params ):
#     n_estimators = params[0]
#     max_depth = params[1]
#     learning_rate = params[2]
#     num_leaves = params[3]
#     min_child_samples = params[4]
#     subsample = params[5]
#     colsample_bytree = params[6]

#     print(params)

#     lgbm_model = lgb.LGBMClassifier( learning_rate = learning_rate, num_leaves=num_leaves, n_estimators=n_estimators, max_depth=max_depth, min_child_samples=min_child_samples, subsample=subsample, colsample_bytree=colsample_bytree, n_jobs=-1, random_state=42, subsample_freq=1)
#     lgbm_model.fit( X_train, y_train )

#     yhat_lgb = lgbm_model.predict( X_val )

#     return -m.f1_score( y_val, yhat_lgb )

# result = gp_minimize( treinar_modelo, search_space, n_calls = 200, n_initial_points = 10, verbose=True, n_jobs=-1, random_state= 42 )
# print('\n')

### 8.1.2 Cross Validation

In [2]:
# LGBM CV BAYESIAN SEARCH

# search_space = [(100, 1500), #name = 'n_estimators'), 
#                 (1, 20), #name = 'max_depth'), 
#                 (0.001, 0.1, 'log-uniform'),#, name = 'learning_rate'),
#                 (2, 128), #name = 'num_leaves'),
#                 (1, 100), #name = 'min_child_samples'),
#                 (0.05, 1.0), #name = 'subsample'),
#                 (0.15, 1.0) #name = 'colsample_bytree')]
# ]

# def treinar_modelo( params ):
#     n_estimators = params[0]
#     max_depth = params[1]
#     learning_rate = params[2]
#     num_leaves = params[3]
#     min_child_samples = params[4]
#     subsample = params[5]
#     colsample_bytree = params[6]

#     print(params, '\n' )

#     lgbm_model = lgb.LGBMClassifier( learning_rate = learning_rate, num_leaves=num_leaves, n_estimators=n_estimators, max_depth=max_depth, min_child_samples=min_child_samples, subsample=subsample, colsample_bytree=colsample_bytree, n_jobs=-1, random_state=42, subsample_freq=1)
#     kfold = KFold(n_splits=5, random_state=1, shuffle=True)
#     cv = cross_val_score(lgbm_model, X, Y, scoring='f1', cv=kfold, n_jobs=-1)

#     return -np.mean(cv)

# result = gp_minimize( treinar_modelo, search_space, n_calls = 200, n_initial_points = 10, verbose=True, n_jobs=-1, random_state= 42 )

## 8.2 CatBoost

### 8.2.1 Normal

In [ ]:
# search_space = [(6, 10), #depth
#                 (0.01, 0.1), #learning rate
#                 (100, 200), #iterations
# ]

# def treinar_modelo( params ):
#     depth = params[0]
#     learning_rate = params[1]
#     iterations = params[2]

#     print(params, '\n' )

#     catboost_model = CatBoostClassifier( depth = depth, learning_rate=learning_rate, iterations=iterations, verbose=False )
#     catboost_model.fit( X_train, y_train )

#     yhat_catboost = catboost_model.predict( X_val )

#     return -m.f1_score( y_val, yhat_catboost )

# result = gp_minimize( treinar_modelo, search_space, n_calls = 100, n_initial_points = 10, verbose=True, n_jobs=-1, random_state= 42 )